In [3]:
import pickle as pkl
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

import sys
sys.path.insert(1, '../')
from encoding import prefix_bin
import numpy as np
import json
import os

In [4]:
def trueTo1(x):
    y_true = []
    for i in x:
        if i =='True':
            y_true.append(1)
        else:
            y_true.append(0)
    return y_true

In [5]:
def return_no_0(y, evaluationlist):
    y_0 =[]
    eval_0=[]
    
    for pos, i in enumerate(evaluationlist):
        if i != 0.0:
            y_0.append(y[pos])
            eval_0.append(i)
            
    return y_0, eval_0

In [29]:
evaluation_dict = {}
result_dict = {}
result_no_avg_dict = {}
evaluation = 'aggregated'
dataset_label = 'OIR5K'
training_rebalance = True
coverage_df = {}

for classifier in ['hatc']:
    with open('../dataset_parameters.json','r') as json_file:
        parameters = json.load(json_file)[dataset_label]
        print(parameters)
        key_pair = parameters['key_pair']
        catatars = parameters['categorical_attrs']
        maximum_prefixs = parameters['maximum_prefix']
        

    for prefix in [1]:
        print(prefix)
        evaluation_dict[prefix] = {}
        result_dict[prefix] = {}

        training_data = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s training window label retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)
        test_data = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s test window label retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)
        result = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s update label retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)

        with open(training_data, 'rb') as openfile:
            training_data = pkl.load(openfile)

        with open(test_data, 'rb') as openfile:
            test_data = pkl.load(openfile)

        with open(result, 'rb') as openfile:
            result = pkl.load(openfile)

        training_label = {'True':0, 'False':0}
        training_data = training_data[200][0]
    #     if classifier == 'hatc':
    #         training_data = training_data[200]
    #     else:
    #         training_data = training_data[200][0]
        for t in training_data:
            label = t.true_label
            training_label[label] +=1

        testing_label={}
        for updates in test_data.keys():
            testing_label[updates] = {'True':0, 'False':0}
            for t in test_data[updates]:
                label = t.true_label
                testing_label[updates][label]+=1

        evaluationlist = {'fscore':[], 'precision':[], 'recall':[]}


        for updates in result[prefix].keys():
            if classifier == 'hatc':
                y_pred = [max(i, key=i.get) for i in result[prefix][updates]['y_pred']]
                y_pred = trueTo1(y_pred)
            else:
                y_pred = [np.argmax(i[0]) for i in result[prefix][updates]['y_pred']]

            y_true = trueTo1(result[prefix][updates]['y_true'])

            evaluationlist['fscore'].append(f1_score(y_true = y_true, y_pred = y_pred, average='weighted'))
            evaluationlist['precision'].append(precision_score(y_true = y_true, y_pred = y_pred, average='weighted'))
            evaluationlist['recall'].append(recall_score(y_true = y_true, y_pred = y_pred, average='weighted'))

        df = pd.DataFrame.from_dict(testing_label).T
        y = [i/30 for i in list(df['True'])]

        for i in evaluationlist:
            y_, eval_ = return_no_0(y, evaluationlist[i])
            y_delta = [y_[pos]-y[pos-1] for pos in range(len(y_)) if pos !=0]
            eval_delta = [eval_[pos]-eval_[pos-1] for pos in range(len(eval_)) if pos !=0]
            result_dict[prefix][i] = np.average(evaluationlist[i])
            evaluation_dict[prefix][i] = np.corrcoef(y_, eval_)[0][1]
            
        coverage_df[prefix] = y

        try:
            os.makedirs('../result/%s/%s/performance'%(dataset_label, classifier))
        except:
            pass
        
        perf_df = pd.DataFrame.from_dict(evaluationlist)
        perf_df_dir = '../result/%s/%s/performance/rebalanced_%s prefix_%s performance label retrained.csv'%(dataset_label, classifier, str(training_rebalance), str(prefix))
        perf_df.to_csv(perf_df_dir, index=False)

    coverage_df = pd.DataFrame.from_dict(coverage_df)
    
    eval_df = pd.DataFrame.from_dict(evaluation_dict)
    result_df = pd.DataFrame.from_dict(result_dict)
    
    coverage_df_dir = '../result/%s/%s/rebalanced_%s label label retrained.csv'%(dataset_label, classifier, str(training_rebalance))
    coverage_df.to_csv(coverage_df_dir, index=False)

    eval_df_dir = '../result/%s/%s/rebalanced_%s label corr label retrained.pkl'%(dataset_label, classifier, str(training_rebalance))
    result_df_dir = '../result/%s/%s/rebalanced_%s perf label retrained.pkl'%(dataset_label, classifier, str(training_rebalance))
    with open(eval_df_dir, 'wb') as r:
        pkl.dump(eval_df, r)

    with open(result_df_dir, 'wb') as r:
        pkl.dump(result_df, r)

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}
1


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 

In [19]:
pd.DataFrame.from_dict(evaluationlist)

,fscore,precision,recall
0,0.757576,0.694444,0.833333
1,0.757576,0.694444,0.833333
2,0.757576,0.694444,0.833333
3,0.757576,0.694444,0.833333
4,0.757576,0.694444,0.833333
...,...,...,...
789,0.620513,0.537778,0.733333
790,0.665409,0.587778,0.766667
791,0.620513,0.537778,0.733333
792,0.620513,0.537778,0.733333


In [13]:
for pos, i in enumerate(coverage_df.loc[:,5]):
    if i <=0.1:
        print(i, pos)

0.1 236
0.1 237
0.1 238
0.1 405
0.1 406
0.1 407
0.1 409
0.1 410
0.1 411
0.1 412
0.1 413
0.1 414
0.1 415
0.1 416
0.06666666666666667 417
0.1 418
0.1 419
0.1 517
0.1 518
0.06666666666666667 519
0.06666666666666667 520
0.03333333333333333 521
0.03333333333333333 522
0.06666666666666667 523
0.06666666666666667 524
0.06666666666666667 525
0.06666666666666667 526
0.06666666666666667 527
0.06666666666666667 528
0.1 529
0.1 530
0.1 531
0.1 532
0.1 533
0.1 534
0.1 535
0.1 536
0.1 733
0.1 734
0.1 735
0.1 736
0.06666666666666667 737
0.06666666666666667 738
0.06666666666666667 739
0.06666666666666667 740
0.06666666666666667 741
0.06666666666666667 742
0.06666666666666667 743
0.06666666666666667 744
0.06666666666666667 745
0.1 746
0.1 747
0.06666666666666667 748
0.1 749
0.1 750
0.1 751
0.1 752
0.1 753
0.1 754
0.1 755
0.1 759


In [14]:
y

[0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.2,
 0.2,
 0.2,
 0.2,
 0.16666666666666666,
 0.2,
 0.23333333333333334,
 0.26666666666666666,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.26666666666666666,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.23333333333333334,
 0.26666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 0.3,
 0.26666666666666666,
 0.26666666666666666,
 0.3,
 0.3,
 0.3,
 0.26666666666666666,
 0.26666666666666666,
 0.23333333333333334,
 0.26666666666666666,
 0.3,
 0.3,
 0.26666666666666666,
 0.26666666666666666,
 0.3,
 0.3,
 0.3,
 0.3,
 0.26666666666666666,
 0.26666666666666666,
 0.23333333333333334,
 0.23333333333333334,
 0.26666

In [13]:
eval_df

,4
fscore,-0.792681
precision,-0.755826
recall,-0.657754
